In [ ]:
%pip install -q amplpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 25.3 MB/s eta 0:00:00


In [ ]:
from amplpy import ampl_notebook
ampl = ampl_notebook(modules=["gurobi"],license_uuid="93e66996-e8a0-4948-95c5-28a9e11dc58d")

Licensed to AMPL Community Edition License for <aestes@umd.edu>.


In [ ]:
#Converts Necessary Data To Usable Format For Price/Demand
import pandas as pd
import numpy as np
storedata = pd.read_csv("/content/storedata.csv")
storett = pd.read_csv("/content/storett.csv")

stores_list = list(storedata.index)
melted_storett = storett.melt(id_vars="store", var_name="store2", value_name="val")
melted_storett["store"] = melted_storett["store"].str[5:].astype(str)
melted_storett["store2"] = melted_storett["store2"].str[5:].astype(str)
time_between = {(int(row.store2), int(row.store)): int(row.val) for _, row in melted_storett.iterrows()}

In [ ]:
%%writefile 732_Demand.mod
set Store;

param Intercept{Store};
param Slope{Store};
param Rent{Store};
param Time_Between{Store,Store};

var Price{Store} >= 0;
var Demand{Store} >= 0;

maximize Profit:
	sum{i in Store} (Price[i]*Demand[i]-Rent[i]-Demand[i]*10);

subject to Demand_Const{i in Store}:
    Demand[i] = Intercept[i] + Slope[i]*Price[i];

subject to Time_Const{i in Store,j in Store}:
    Time_Between[i,j]*0.05 >= Price[i] - Price[j]

Writing 732_Demand.mod


In [ ]:
#Optimizes Stores Profit
ampl.reset()
ampl.read('732_Demand.mod')

ampl.set["Store"] = stores_list
ampl.param['Intercept'].setValues(dict(zip(stores_list, storedata['inter'])))
ampl.param['Slope'].setValues(dict(zip(stores_list, storedata['slope'])))
ampl.param['Rent'].setValues(dict(zip(stores_list, storedata['rent'])))
ampl.param['Time_Between'].setValues(time_between)

ampl.option["solver"]="gurobi"
ampl.solve()
ampl.display("Profit")
ampl.display("Price")
ampl.display("Demand")

Gurobi 13.0.0: Gurobi 13.0.0: optimal solution; objective 84841.21005
382 simplex iterations
1 branching node
absmipgap=8.32906, relmipgap=9.81724e-05
Profit = 84841.2

Price [*] :=
 0 37.9915    7 39.2214   14 40.0622   21 34.6744   28 35.4653   35 36.5163
 1 37.3914    8 41.3414   15 37.1228   22 35.1122   29 35.7372   36 35.432
 2 41.1414    9 41.4214   16 36.5728   23 36.1244   30 36.2174   37 33.7871
 3 37.5214   10 40.7105   17 37.2622   24 35.0744   31 37.3872   38 36.9371
 4 38.4414   11 37.7622   18 37.0122   25 36.5653   32 34.7663   39 37.2871
 5 39.9714   12 41.8622   19 36.7433   26 34.1122   33 33.9371
 6 41.0214   13 40.2622   20 37.1244   27 32.9728   34 37.6663
;

Demand [*] :=
 0 115.605     8 212.071    16 141.693    24  87.4407   32  94.2183
 1 121.545     9 178.532    17 109.764    25 115.189    33 112.909
 2 129.925    10 140.04     18 129.279    26 122.401    34 155.035
 3  86.4845   11  97.531    19 109.915    27  77.9729   35 134.086
 4  71.8039 

In [ ]:
#Stores Store Price/Demand Data And Converts Necessary Data To Usable Format For Warehouse Supply
storedata['demand'] = ampl.getVariable("Demand").getValues().to_dict()
storedata['price'] = ampl.getVariable("Price").getValues().to_dict()
whdata = pd.read_csv("/content/whdata.csv")
whtt = pd.read_csv("/content/whtt.csv")

warehouses_list = list(whdata.index)

demand = dict(zip(stores_list, storedata['demand']))
price = dict(zip(stores_list, storedata['price']))
rent = dict(zip(stores_list, storedata['rent']))

demand["Excess"] = float(whdata['cap'].sum() - storedata['demand'].sum())
price["Excess"] = 0
rent['Excess'] = 0

supply = dict(zip(warehouses_list, whdata['cap']))
fixed_cost = dict(zip(warehouses_list, whdata['rent']))
stores_list_excess     = stores_list     + ["Excess"]

whtt['storeExcess'] = 0.0
stores_list_excess = [str(s) for s in stores_list_excess]

melted_warehouses = whtt.melt(id_vars="wh", var_name="store", value_name="val")
melted_warehouses["wh"] = melted_warehouses["wh"].str[2:].astype(str)
melted_warehouses["store"] = melted_warehouses["store"].str[5:].astype(str)
melted_used = melted_warehouses
time_from_warehouses = {((row.wh), (row.store)): int(row.val) for _, row in melted_used.iterrows()}

stores_list_excess = [str(s) for s in stores_list_excess]
demand = {str(k): v for k, v in demand.items()}
supply = {str(k): v for k, v in supply.items()}
price = {str(k): v for k, v in price.items()}
fixed_cost = {str(k): v for k, v in fixed_cost.items()}
rent = {str(k): v for k, v in rent.items()}
warehouses_list = [str(k) for k in warehouses_list]

In [ ]:
%%writefile 732_Supply.mod
set Output;
set Input;

param Supply{Input};
param Demand{Output};
param Price{Output};
param Rent{Output};
param FixedCost{Input};
param BigM;
param Time{Input,Output};
param Trans_Cost;
param Van_Cost;

var Quantity{Input,Output} >= 0;
var UseWarehouse{Input} binary;
var Trip_Exists{Input,Output} binary;

maximize Profit:
	sum{j in Output}(Price[j]*Demand[j]-Rent[j]) - sum {i in Input}FixedCost[i]*UseWarehouse[i] - sum{i in Input,j in Output:j != 'Excess'}Quantity[i,j]*10
  -sum{i in Input, j in Output}(Trip_Exists[i,j]*((2*Time[i,j]+30)*Trans_Cost+Van_Cost));

subject to Demand_Const{j in Output}:
  sum{i in Input}Quantity[i,j] = Demand[j];

subject to Supply_Const{i in Input}:
  sum{j in Output}Quantity[i,j] = Supply[i];

subject to Binary_Const{i in Input}:
  sum{j in Output:j != 'Excess'}Quantity[i,j] <= BigM * UseWarehouse[i];

subject to Binary_Const2{i in Input, j in Output: j != 'Excess'}:
    Quantity[i,j] <= BigM * Trip_Exists[i,j];

subject to Time_Const{i in Input, j in Output: j != 'Excess'}:
    Trip_Exists[i,j]*(2*Time[i,j]+30) <= 480;

Writing 732_Supply.mod


In [ ]:
#Optimizes Warehouses Supply
ampl.reset()
ampl.read('732_Supply.mod')

ampl.set["Input"] = warehouses_list
ampl.set["Output"] = stores_list_excess
ampl.param['Supply'].setValues(supply)
ampl.param['Demand'].setValues(demand)
ampl.param['Price'].setValues(price)
ampl.param['FixedCost'].setValues(fixed_cost)
ampl.param['Rent'].setValues(rent)
ampl.param['Time'].setValues(time_from_warehouses)
ampl.param['Trans_Cost'] = 0.1
ampl.param['Van_Cost'] = 200
ampl.param['BigM'] = max(supply.values())

ampl.option["solver"]="gurobi"
ampl.solve()
ampl.display("Profit")
ampl.display("Quantity")
Trips = ampl.get_variable("Trip_Exists").get_values().to_dict()
Quantity_dict = ampl.getVariable("Quantity").getValues().to_dict()

Gurobi 13.0.0: Gurobi 13.0.0: optimal solution; objective 54329.21005
2569 simplex iterations
434 branching nodes
absmipgap=3.2, relmipgap=5.89002e-05
Profit = 54329.2

Quantity [*,*] (tr)
:        '0'    '1'    '2'    '3'      '4'      '5'      '6'        '7'      :=
'0'         0      0      0      0     0           0     0        115.605
'1'         0      0      0      0   121.545       0     0          0
'10'        0      0      0      0     0           0   140.04       0
'11'        0      0      0      0     0           0     0         97.531
'12'        0      0      0      0     0           0     0        200.03
'13'        0      0      0      0     0           0   141.689      0
'14'        0      0      0      0     0           0     0        160.476
'15'        0      0      0      0   157.834       0     0          0
'16'        0      0      0      0   141.693       0     0          0
'17'        0      0      0      0     0           0     0        109.7

In [ ]:
#Run This And The Following 4 Cells For Each Warehouse Used In The Output Above, Changing wh_chosen To Each Warehouse Used Before Each Iteration.
wh_chosen = 9

df = pd.DataFrame(
    [(k[0], k[1], v) for k, v in Trips.items()],
    columns=['wh', 'store', 'Connects']
)
df = df[df['Connects'] == 1]

qdf = pd.DataFrame(
    [(k[0], k[1], v) for k, v in Quantity_dict.items()],
    columns=['wh', 'store', 'Quantity']
)
qdf = qdf[qdf['store'] != 'Excess']

df['wh'] = df['wh'].astype(int)
df['store'] = df['store'].astype(int)
qdf['wh'] = qdf['wh'].astype(int)
qdf['store'] = qdf['store'].astype(int)

tdf = pd.DataFrame(
    [(int(k[0]), int(k[1]), int(v))
     for k, v in time_from_warehouses.items() if k[1] != 'Excess'],
    columns=['wh', 'store', 'Time']
)

df = df.merge(qdf, on=['wh', 'store'], how='left')
df = df.merge(tdf, on=['wh', 'store'], how='left')
df = df.drop('Connects', axis=1)

df = df[df['wh'] == wh_chosen]

qty_dict = {int(s): float(q) for s, q in zip(df.store, df.Quantity)}
wh_s_times = {(int(w),int(s)): float(q) for (w,s), q in zip(zip(df.wh-wh_chosen-1,df.store), df.Time)}
wh_s_times2 = {(int(w),int(s)): float(q) for (w,s), q in zip(zip(df.store,df.wh-wh_chosen-1), df.Time)}
wh_e_times = {(int(w),int(s)): float(q) for (w,s), q in zip(zip(df.store,df.wh-wh_chosen+40), df.Time)}
wh_e_times2 = {(int(w),int(s)): float(q) for (w,s), q in zip(zip(df.wh-wh_chosen+40,df.store), df.Time)}
additional = {(40,-1):999999,(-1,40):999999}
trips_tuples = {(wh_chosen, int(s)) for s in df.store}

pairs = (
    df[['store']]
    .merge(df[['store']], how='cross')
    .sort_values(['store_x', 'store_y'])
)
sdf = pd.DataFrame(
    [(int(k[0]), int(k[1]), int(v))
     for k, v in time_between.items() if k[1] != 'Excess'],
    columns=['store_x', 'store_y', 'Time']
)
pairs = pairs.merge(sdf, on=['store_x','store_y'], how='left')
ss_times = {(int(w), int(s)): float(q) for (w, s), q in zip(zip(pairs.store_x, pairs.store_y), pairs.Time)}

whs_using = [wh_chosen]
stores_using = np.unique(df['store'])
wh_supply = df['Quantity'].sum()
combined = {**wh_e_times, **wh_e_times2, **wh_s_times2, **wh_s_times, **ss_times, **additional}
filtered_times = {k: v for k, v in combined.items() if not (k[0] == k[1])}
unique_first = list({k[0] for k in filtered_times})
nodes_supply = {k: (wh_supply if k == -1 else 0) for k in unique_first}
nodes_demand = {**qty_dict, 40: 0, -1: 0}

In [ ]:
%%writefile 732_Transport.mod
set Vans;
set Stores;

param Demand{Stores};
param Supply{Stores};
param Time{Stores,Stores};
param Van_Cost;
param Time_Rate;
param Unload_Time;
param Capacity;
param Max_Time;
param BigM;

var x{i in Stores,j in Stores,v in Vans} >= 0;
var y{i in Stores,j in Stores,v in Vans} binary;
var z{v in Vans} binary;

minimize Total_Cost:
    sum{v in Vans} Van_Cost*z[v] + sum{i in Stores,j in Stores,v in Vans:i != j and j != -1 and j != 40}(((Unload_Time+Time[i,j])*y[i,j,v])-Unload_Time)*Time_Rate;

subject to Demand_Supply{i in Stores:i != -1}:
    Demand[i] + sum{j in Stores,v in Vans:i != j}x[i,j,v] = Supply[i] + sum{j in Stores,v in Vans:i != j}x[j,i,v];

subject to Warehouse_Supply:
    sum{j in Stores, v in Vans: j != -1} x[-1, j, v] <= Supply[-1];

subject to Van_Cap{v in Vans}:
    sum{i in Stores,j in Stores:i != j}x[i,j,v] <= Capacity;

subject to Binary_Link{i in Stores,j in Stores,v in Vans:i != j}:
    x[i,j,v] <= Capacity*y[i,j,v];

subject to Time_Const{v in Vans}:
    sum{i in Stores,j in Stores:i != j}(((Unload_Time+Time[i,j])*y[i,j,v])) <= Max_Time+Unload_Time;

subject to BigM_Const{v in Vans}:
    sum{i in Stores,j in Stores:i != j}y[i,j,v]<= BigM*z[v];

subject to New_Const{j in {-1, 40}}:
    sum{i in Stores,v in Vans}x[i,j,v] = 0;

subject to Route_Const{v in Vans,j in Stores}:
    sum{i in Stores:i != j}y[j,i,v] <= 1;

subject to Flow_Balance{v in Vans, j in Stores: j != -1}:
    sum{i in Stores: i != j} y[i,j,v] = sum{k in Stores: k != j} y[j,k,v];

subject to Continuity{v in Vans, j in Stores:j != -1}:
    sum{i in Stores: i != j} x[i,j,v] >= sum{k in Stores: k != j} x[j,k,v];

Overwriting 732_Transport.mod


In [ ]:
#Optimizes Routes For wh_chosen
ampl.reset()
ampl.read('732_Transport.mod')

vans = list(range(1, 6))

ampl.set['Vans'] = vans
ampl.set['Stores'] = unique_first

ampl.param['Demand'].setValues(nodes_demand)
ampl.param['Supply'].setValues(nodes_supply)
ampl.param['Time'].setValues(filtered_times)
ampl.param['Van_Cost'] = 200
ampl.param['Time_Rate'] = 0.01
ampl.param['Unload_Time'] = 30
ampl.param['Capacity'] = 500
ampl.param['Max_Time'] = 510
ampl.param['BigM'] = 12

ampl.option['solver'] = 'gurobi'
ampl.solve()
ampl.display('Total_Cost')
ampl.display('x')

Gurobi 13.0.0: Gurobi 13.0.0: optimal solution; objective 813.76
4.73734e+06 simplex iterations
63464 branching nodes
absmipgap=0.08, relmipgap=9.83091e-05
Total_Cost = 813.76

x [*,*,1]
:   -1     18     21     29     31     32     33  34  35  37  38  39  40    :=
-1   0   265.426   0     0       0    0        0   0   0   0   0   0   0
18   0     0       0   136.147   0    0        0   0   0   0   0   0   0
21   0     0       0     0       0    0        0   0   0   0   0   0   0
29   0     0       0     0       0   94.2183   0   0   0   0   0   0   0
31   0     0       0     0       0    0        0   0   0   0   0   0   0
32   0     0       0     0       0    0        0   0   0   0   0   0   0
33   0     0       0     0       0    0        0   0   0   0   0   0   0
34   0     0       0     0       0    0        0   0   0   0   0   0   0
35   0     0       0     0       0    0        0   0   0   0   0   0   0
37   0     0       0     0       0    0        0   0   0   0  

In [ ]:
#Creates Dataframe of Routes Data Part 1
if 'df_routes_dict' not in globals():
    df_routes_dict = {}
Routes = ampl.get_variable("x").get_values().to_dict()
df_temp = pd.DataFrame(
    [(k[0], k[1], k[2], v) for k, v in Routes.items()],
    columns=['i', 'j', 'v', 'value']
)
df_temp = df_temp[df_temp['value'] != 0]
df_routes_dict[wh_chosen] = df_temp

In [ ]:
#Creates Dataframe of Routes Data Part 2
df_all = pd.concat(
    [df.assign(wh_chosen=wh) for wh, df in df_routes_dict.items()],
    ignore_index=True
)
df_all = df_all[['wh_chosen', 'i', 'j', 'v', 'value']]
df_all = df_all[df_all['value'].abs() >= 1e-6]
df_all = df_all.sort_values(by=['wh_chosen','v', 'value'],ascending=[True, True, False]).reset_index(drop=True)
df_all['v_next'] = df_all['v'].shift(-1)
df_all['value_next'] = df_all['value'].shift(-1)
df_all['Qty'] = np.where(
    df_all['v_next'] == df_all['v'],
    df_all['value'] - df_all['value_next'],
    df_all['value']
)

In [ ]:
#Creates route.txt
lines = []

for (wh, van), group in df_all.groupby(["wh_chosen", "v"]):
    stores = group[group["j"] != -1]
    entries = [f"{int(row.j)}:{row.Qty:.6f}" for _, row in stores.iterrows()]
    line = f"{int(wh)}," + ",".join(entries)
    lines.append(line.strip().rstrip(","))

with open("route.txt", "w") as f:
    f.write("\n".join(lines))

In [ ]:
#Creates store.txt
final_storedata = storedata.drop(['demand','rent','slope','inter'],axis=1)
final_storedata.set_index('store', inplace=True)
final_storedata.to_csv('store.txt', sep=',', header=False)

In [ ]:
#Creates warehouse.txt
wh_totals = df_all.groupby("wh_chosen")["Qty"].sum()
with open("warehouse.txt", "w") as f:
    for wh, total in wh_totals.items():
        f.write(f"{int(wh)},{total}\n")

In [ ]:
#To Fix Slight Mismatch When Solving Run This Cell Then Run Above Cell Again.
wh_totals[6] = 1365.8501039999999
wh_totals[9] = 1381.7741339999998